In [44]:
import tensorflow as tf
import torch
from transformers import BertTokenizer, TFBertModel, BertModel

In [45]:
bert_type='cimm-kzn/endr-bert'

In [46]:
import tensorflow as tf
tf.executing_eagerly()

True

In [47]:
tokenizer_bert = BertTokenizer.from_pretrained(bert_type)
model = BertModel.from_pretrained(bert_type)

In [48]:
text_to_tokenize = "This is text that I want to tokenize, but I also do it with span inside"

In [49]:
span = (18, 36)

# Using Torch for this model

In [50]:
tokenizer = BertTokenizer.from_pretrained(bert_type)
model = BertModel.from_pretrained(bert_type)

In [51]:
tokens = tokenizer.encode(text_to_tokenize)
input_ids = torch.tensor(tokens).unsqueeze(0)  # Batch size 1

In [52]:
input_ids

tensor([[  101, 10531, 10124, 15541, 10189,   177, 21528, 10114, 18436, 18687,
         10870,   117, 10473,   177, 10379, 10149, 10271, 10169, 51551, 22978,
           102]])

In [53]:
outputs = model(input_ids)

In [54]:
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple

In [55]:
last_hidden_states.shape

torch.Size([1, 21, 768])

# Vectorize text with span

In [86]:
tokenizer = BertTokenizer.from_pretrained(bert_type)
model = BertModel.from_pretrained(bert_type)

tokens = tokenizer.encode(text_to_tokenize)
input_ids = torch.tensor(tokens).unsqueeze(0)
outputs = model(input_ids)
last_hidden_states = outputs[0]

In [87]:
tokenized_text = tokenizer.tokenize(text_to_tokenize)
tokenized_span = tokenizer.tokenize(text_to_tokenize[18:36])

In [88]:
len(tokenized_text), len(last_hidden_states[0])

(19, 21)

In [116]:
span_words_indices = [i+1 for i in range(len(tokenized_text)) if tokenized_text[i] in tokenized_span][:len(tokenized_span)]
span_words_indices

[5, 6, 7, 8, 9, 10]

In [117]:
tokenizer.decode(input_ids.tolist()[0])

'[CLS] this is text that i want to tokenize, but i also do it with span inside [SEP]'

In [118]:
tokenizer.decode([input_ids.tolist()[0][j] for j in span_words_indices])

'i want to tokenize'

In [121]:
span_ids = [input_ids.tolist()[0][j] for j in span_words_indices]
span_ids

[177, 21528, 10114, 18436, 18687, 10870]

In [123]:
input_ids = torch.tensor(span_ids).unsqueeze(0)
outputs = model(input_ids)
last_hidden_states = outputs[0]

In [134]:
last_hidden_states[0].mean(axis=1)

tensor([ 0.0012, -0.0062, -0.0092, -0.0069, -0.0113, -0.0069],
       grad_fn=<MeanBackward1>)

In [135]:
last_hidden_states[0]

tensor([[ 0.6198, -0.2913, -0.0801,  ...,  0.2206,  0.1563,  0.0305],
        [ 0.4594, -0.7568,  0.0446,  ...,  0.5830,  0.5598, -0.2870],
        [ 1.0232, -0.9621, -0.7415,  ...,  0.4972,  0.4687, -0.0192],
        [ 0.4532, -0.3883, -0.1660,  ..., -0.2376,  0.2915, -0.4778],
        [-0.0521, -0.3073, -0.3796,  ...,  0.0637,  0.3936, -0.2138],
        [ 0.6270, -0.8474, -0.3692,  ...,  0.2373,  0.3379, -0.1057]],
       grad_fn=<SelectBackward>)

In [242]:
def get_vecors_from_context_TORCH(text, span):
    """ Get sent vec with TOCH model """
    if span in [None, ''] or span not in text:
        span = text
    tokenized_text = tokenizer.tokenize(text)
    tokenized_span = tokenizer.tokenize(span)
    text_ids = tokenizer.encode(text)
    # get tokens vecs
    text_ids = torch.tensor(text_ids).unsqueeze(0)
    outputs = model(text_ids)
    last_hidden_states = outputs[0]
    # define which vecs are related to span
    span_words_indices = [
        i+1 for i in range(len(tokenized_text)) if tokenized_text[i] in tokenized_span
    ][:len(tokenized_span)]
    span_vec = np.array(last_hidden_states[0][span_words_indices].tolist())
    print(np.mean(span_vec, axis=0).shape)
    #return span_vec

In [250]:
import numpy as np
get_vecors_from_context_TORCH(
    "severe abdominal cramps, leg cramps, diahhrea, after taking 10mg only 3 days.<SENT>Symptoms still persist 1 week later.<SENT>Don't take it.", "Don't take it.")

(768,)


In [269]:
import pandas as pd

d = pd.read_csv('../../data/interim/cadec/test.csv')
for k in range(d.shape[0]):
    text = d.iloc[k].to_dict()['sent']
    span = d.iloc[k].to_dict()['term']
    try:
        get_vecors_from_context_TORCH(text, span)
    except Exception as e:
        
        print('>> ', text)
        print('>> ', span)
        print('\n', e)
        break

(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)


KeyboardInterrupt: 

In [256]:
pd.read_csv('../../data/interim/cadec/test.csv')

,term,start,end,sent,text,code,STR,SNMS
0,muscle problems,36,44,Absolutely none.,Absolutely none.<SENT>No muscle or joint probl...,10028641,Myopathy,"['Myopathy', 'Myopathy', 'Myopathy', 'Myopathy..."
1,Exhaustion,0,10,"Exhaustion, malaise, upset stomach/constant hu...","Exhaustion, malaise, upset stomach/constant hu...",10015667,Exhaustion,"['Exhaustion', 'Exhaustion', 'Exhaustion', 'Ex..."
2,Brain Fog,9,18,I am diabetic and my numbers weren't horrible ...,"Fatigue, Brain Fog, Stuttering, Bad OCD!.<SENT...",10027371,Mental dullness,"['Mentally dull', 'Mental dullness', 'Mental d..."
3,cramping neck muscels,18,39,"been off it 10 months,drs spent fortune 160 k ...","vertigo caused by cramping neck muscels,all th...",10028334,Muscle spasms,"['Spasm', 'Spasm, NOS', 'Muscle spasm', 'Muscl..."
4,hips hurt,126,135,"Stopped, and pains lessened in one week; took ...",took Lipitor for over a year with no negative ...,10033432,Pain in hip,"['Hip pain', 'Hip pain', 'Hip pain', 'Hip pain..."
...,...,...,...,...,...,...,...,...
876,not being able to concentrate,634,663,"The light headedness and often foggy vision, s...","Persistent triedness,loss of concentration,unc...",10010248,Concentration ability impaired,[]
877,Lots of gas,108,119,Dosage for past 2 yrs has been 40 mg once a day.,"Muscle and joint aches in arms, elbows, and ca...",10016766,Flatulence,"['Flatulence', 'Flatulence', 'Flatulence', 'Fl..."
878,Body Pains,0,10,"Body Pains all over, major on going muscle cra...","Body Pains all over, major on going muscle cra...",10018074,Generalised aching,"['Generalized aches and pains', 'Generalized a..."
879,Constant sleepiness,220,239,"Also, hot/cold fever-like sensations in the sk...",51 yr old female.<SENT>After 2 1/2 years on it...,10020765,Hypersomnia,"['Hypersomnia', 'Hypersomnia, NOS', 'Hypersomn..."
